In [1]:
import re

import pandas as pd

from pathlib import Path

DATA_DIR = Path.cwd().parent.parent / "data"

In [2]:
df = pd.read_csv(DATA_DIR / "scraped_pages-final.csv", usecols=["url", "title", "text"])

In [3]:
df.shape

(13815, 3)

In [4]:
df["url"].nunique()

13815

In [5]:
df.head()

,url,title,text
0,https://downloads.dell.com/manuals/all-product...,NaN,"Dell EMC Networking N-Series\nN1100-ON, N1500,..."
1,https://downloads.dell.com/manuals/all-product...,NaN,"Dell™ SR2220L monitor User's Guide \nNotes, No..."
2,https://downloads.dell.com/manuals/all-product...,NaN,1\n2\n10°\n5°\n2016\n(Dell P/N:VC35H rev.A00)\...
3,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...
4,https://downloads.dell.com/manuals/all-product...,NaN,\nStatement of Volatility – Dell U2419H \n \n...


In [6]:
is_pdf = df["url"].str.endswith("pdf", na=False)
html_df = df.loc[~is_pdf].copy()
pdf_df = df.loc[is_pdf].copy()

print(f"Number of html docs: {len(html_df):,}")
print(f"Number of pdf docs: {len(pdf_df):,}")

Number of html docs: 2,623
Number of pdf docs: 11,192


### HTML text cleaning

In [7]:
html_df.iloc[3,2]

'Skip to main content\nContact Us\nUK/EN\nBasket\nProducts\nSolutions\nServices\nSupport\nSupport Home\nProduct Support\nSupported Operating Systems\u200b\nSupported Operating Systems\u200b\nYour TV Tuner RDVBT supports these operating systems:\nChange Product\nWindows 7, 32-bit\nWindows 7, 64-bit\nWindows Vista, 32-bit\nWindows Vista, 64-bit\nWindows XP\nNote regarding operating systems not listed above:\nDell may not support an operating system for various reasons, including discontinued support from the software vendor, lack of availability with certain products, or other reasons.\nUK/EN\nSite Map\nAccount\nMy Account\nOrder Status\nProfile Settings\nMy Products\nDell Rewards Balance\nSupport\nSupport Home\nContact Technical Support\nReturns\nConnect with Us\nCommunity\nContact Us\nOur Offerings\nArtificial Intelligence\nProducts\nSolutions\nServices\nDeals\nOur Company\nWho We Are\nCareers\nDell Technologies Capital\nInvestors\nNewsroom\nRecycling\nCorporate Impact\nCustomer Storie

In [8]:
partition = re.compile(r"(?:\n+|(?<=[.!?])\s+)")
pre_clean = lambda text: re.sub(
    r"[ \t]+", " ",
    re.sub(r" *\n *", "\n",
           str(text).replace("\r\n", "\n").replace("\r", "\n").replace("\u200b", " ").replace("\xa0", " ")
    ).strip().lower()
)

html_df["cleaned_text"] = html_df["text"].apply(lambda t: t if pd.isna(t) else re.sub(r"\s*\|\s*Dell UK\s*", " ", pre_clean(t)).strip())
html_df["cleaned_title"] = html_df["title"].apply(lambda t: t if pd.isna(t) else pre_clean(t).replace("\n", " "))

doc_sents = html_df["cleaned_text"].dropna().map(
    lambda text: {s.strip() for s in partition.split(text) if s.strip()}
)
n_docs = len(doc_sents)

common_html_df = (doc_sents.explode().value_counts() / n_docs).reset_index()
common_html_df.columns = ["sentence", "proportion"]
common_html_df = common_html_df.sort_values(["proportion", "sentence"], ascending=[False, True]).reset_index(drop=True)

common_html_df.head(70)

,sentence,proportion
0,accessibility,1.000000
1,account,1.000000
2,"anti-slavery, human trafficking & child labor",1.000000
3,artificial intelligence,1.000000
4,basket,1.000000
...,...,...
65,"windows vista, 32-bit",0.235227
66,windows xp,0.229508
67,"windows vista, 64-bit",0.195959
68,"windows 8, 32-bit",0.189859


In [9]:
sentences_to_delete = common_html_df.loc[common_html_df["proportion"] == 1., "sentence"]

In [10]:
sentences_to_delete = set(common_html_df.loc[common_html_df["proportion"] == 1., "sentence"])

html_df["cleaned_text"] = html_df["cleaned_text"].apply(
    lambda t: t if pd.isna(t) else "\n".join(
        s.strip() for s in partition.split(t) if s.strip() and s.strip() not in sentences_to_delete
    )
)

In [11]:
html_df.iloc[0,4]

'supported operating systems | dell uk'

In [12]:
html_df["len_text"] = html_df["cleaned_text"].str.len()

In [13]:
html_df[["url", "len_text"]].nlargest(50, "len_text")

,url,len_text
6526,https://www.dell.com/support/contents/en-uk/ar...,15075
1016,https://www.dell.com/support/contents/en-uk/ar...,13800
8967,https://www.dell.com/support/contents/en-uk/ar...,12129
4140,https://www.dell.com/support/contents/en-uk/ar...,11657
1757,https://www.dell.com/support/contents/en-uk/ar...,11207
11666,https://www.dell.com/support/contents/en-uk/ar...,11047
3955,https://www.dell.com/support/contents/en-uk/ar...,9864
5797,https://www.dell.com/support/contents/en-uk/ar...,9758
6080,https://www.dell.com/support/contents/en-uk/ar...,9651
1096,https://www.dell.com/support/contents/en-uk/ar...,9433


In [14]:
finding_product_mask = html_df["url"].fillna("").str.contains(r"/supportedos/", regex=True)

html_df["product"] = "general"
html_df.loc[finding_product_mask, "product"] = html_df.loc[finding_product_mask, "url"].str.extract(r"/supportedos/([^/?#]+)", expand=False)

html_df["product"] = html_df["product"].fillna("general")

In [15]:
html_df.head()

,url,title,text,cleaned_text,cleaned_title,len_text,product
3,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,447,inspiron-15-5545-laptop
9,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,418,studio-1435
12,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,509,poweredge-2321ds
28,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,463,dgtvrdvbt-01
37,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,410,studio-xps-7100


In [16]:
html_df["product"].value_counts()

product
general                        135
poweredge-1850                   1
optiplex-9030-aio                1
inspiron-20-3052-aio             1
powervault-112t                  1
                              ... 
xps-710-h2c                      1
precision-t7500                  1
precision-t3420-workstation      1
inspiron-500m                    1
optiplex-xm5                     1
Name: count, Length: 2489, dtype: int64

In [17]:
html_df[html_df["product"].str.contains("inspiron")]

,url,title,text,cleaned_text,cleaned_title,len_text,product
3,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,447,inspiron-15-5545-laptop
74,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,419,inspiron-15-7577-laptop
86,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,409,inspiron-5675-gaming-desktop
130,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,387,inspiron-500m
135,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,442,inspiron-20-3052-aio
...,...,...,...,...,...,...,...
13587,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,518,inspiron-15-5552-laptop
13619,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,412,inspiron-17-5765-laptop
13692,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,415,inspiron-15-7548-laptop
13737,https://www.dell.com/support/home/en-uk/driver...,Supported Operating Systems​ | Dell UK,Skip to main content\nContact Us\nUK/EN\nBaske...,product support\nsupported operating systems\n...,supported operating systems | dell uk,466,inspiron-546


In [18]:
pdf_df.isna().sum()/len(pdf_df)

url      0.000000
title    1.000000
text     0.080147
dtype: float64

In [19]:
pdf_df = pdf_df.dropna(subset=["text"])

In [20]:
pdf_df["url"].head()

0    https://downloads.dell.com/manuals/all-product...
1    https://downloads.dell.com/manuals/all-product...
2    https://downloads.dell.com/manuals/all-product...
4    https://downloads.dell.com/manuals/all-product...
5    https://downloads.dell.com/manuals/all-product...
Name: url, dtype: object

In [21]:
pdf_df["product"] = pdf_df["url"].str.extract(r"/([^/_]+)(?:_[^/]*)?\.pdf$", expand=False)

In [22]:
pdf_df["product"].value_counts()

product
high-computing-solution-resources     102
s-solution-resources                   99
powerconnect-w-airwave                 66
poweredge-m1000e                       62
dell-basic-pdu                         41
                                     ... 
platespin                               1
wyse-d00dx                              1
dell-command-integration-suite-v50      1
vostro-14-3459-laptop                   1
latitude-cport                          1
Name: count, Length: 3015, dtype: int64

In [23]:
pdf_set = set(pdf_df["product"].unique())
html_set = set(html_df["product"].unique())

print(f"Number of products identified in web pages: {len(html_set):,}")
print(f"Number of products identified in pdf docs: {len(pdf_set):,}\n")
print(f"Number of products identified in pdf docs but not in web pages: {len(pdf_set - html_set):,}")
print(f"Number of products identified in web pages but not in pdf docs: {len(html_set - pdf_set):,}")
print(f"Number of products identified in both web pages and pdf docs: {len(html_set & pdf_set):,}")

Number of products identified in web pages: 2,489
Number of products identified in pdf docs: 3,015

Number of products identified in pdf docs but not in web pages: 1,180
Number of products identified in web pages but not in pdf docs: 654
Number of products identified in both web pages and pdf docs: 1,835


Upon visual inspection, there are not obvious errors/misspelings, but simply different products, and sometimes general support/help pages.

In [24]:
pdf_df.loc[pdf_df["product"].isin(pdf_set - html_set), "product"].value_counts()

product
high-computing-solution-resources      102
s-solution-resources                    99
powerconnect-w-airwave                  66
dell-basic-pdu                          41
powerconnect-w-iap-105                  39
                                      ... 
powerconnect-j-ex4500                    1
dell-active-pen-pn338m                   1
dell-cmnd-mntr-v9.1                      1
dell-remote-access-config-tool-v1.0      1
dell-remote-access-cntrllr-5-v1.45       1
Name: count, Length: 1180, dtype: int64

In [25]:
html_df.loc[html_df["product"].isin(html_set - pdf_set), "product"].value_counts()

product
general                                    135
oth-486-mdt-3                                1
oth-r740-xl                                  1
dell-edge-gateway-3000-series-oem-ready      1
g-series-15-5505-laptop                      1
                                          ... 
vostro-14-3446                               1
poweredge-4100                               1
alienware-m17-laptop                         1
inspiron-15-3565-laptop                      1
inspiron-5675-gaming-desktop                 1
Name: count, Length: 654, dtype: int64

In [26]:
pdf_df.iloc[0,2]

'Dell EMC Networking N-Series\nN1100-ON, N1500, N2000,\nN2100-ON, N2200-ON, N3000-\nON, and N3100-ON Switches\nUser’s Configuration\nGuide\nVersion 6.6.1\nRegulatory Models: E04W, E05W, E06W, \nE07W, E15W, E16W, E17W, E18W, E41W\nNotes and Cautions\n NOTE: A NOTE indicates important information that helps you make better use of \nyour computer.\n CAUTION: A CAUTION indicates potential damage to hardware or loss of data if \ninstructions are not followed.\n \n____________\nInformation in this publication is subject to change without notice.\nCopyright © 2019 Dell EMC Inc. All rights reserved. \nReproduction of these materials in any manner whatsoever without the written permission of Dell Inc. \nis strictly forbidden.\nThis product is protected by U.S. and international copyright and intellectual property laws. Dell \nEMC™ and the Dell EMC logo are trademarks of Dell EMC Inc. in the United States and/or other \njurisdictions. All other marks and names mentioned herein may be trademarks 

In [27]:
html_df = html_df.drop(["text", "title", "len_text"], axis=1)
html_df = html_df.rename({"cleaned_text":"text", "cleaned_title":"title"}, axis=1)

In [28]:
html_df.columns, pdf_df.columns

(Index(['url', 'text', 'title', 'product'], dtype='object'),
 Index(['url', 'title', 'text', 'product'], dtype='object'))

In [29]:
df_to_embed = pd.concat([html_df, pdf_df], ignore_index=True)
df_to_embed.shape

(12918, 4)

In [30]:
df_to_embed["product"].value_counts()

product
general                                    135
high-computing-solution-resources          102
s-solution-resources                        99
powerconnect-w-airwave                      66
poweredge-m1000e                            63
                                          ... 
dell-remote-access-config-tool-v1.0          1
dell-remote-access-cntrllr-5-v1.45           1
drseriesrelease%20notes2                     1
dell-powervault-md3000-rh-ent-lx-ha-ctr      1
printergraphicssymbols                       1
Name: count, Length: 3669, dtype: int64

In [36]:
df_to_embed["product"].drop_duplicates().to_csv(DATA_DIR / "product_list.csv", index=False)